In [1]:
import pandas as pd
import numpy as np
import math
import calendar
import os

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse 
sns.set()

from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold

from sklearn.base import clone


from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm


import warnings
warnings.filterwarnings("ignore")

In [2]:
# Здесь будем делать прогноз на k-периодов (месяцев)

In [3]:
# Сделаем функцию по генерации признаков

def create_pred_features(data, period = 12):
    """
    в данной функции генерируем признаки на период предсказания
    На вход принимаем датафрейм на базе которого обучаем модель
    """
    data = pd.DataFrame(data)
    # добавляем столбец с датой, перводим в datetime и отправляем в индекс
    data.columns = ['date', 'PO', 'temp_C']
    data['date'] = pd.to_datetime(data['date'], dayfirst = True)
    data.index = data['date']
    
    # добавляем номер месяца
    data['month'] = data.index.month
    # добавляем столбец с годом
    data['year'] = data.index.year
    # добавляем cos(месяца)
    data['cos_month'] = np.cos((data['month']-1) / 12 * 2 * np.pi)

    # добавим столбцы со скользящими средними
    data['po_average2'] = data['PO'].rolling(window =2).mean()
    data['po_average2'][0] = data['PO'][0]
    data['po_average_exp2'] = data['PO'].ewm(min_periods=2, span=2).mean()
    data['po_average_exp2'][0] = data['PO'][0]


    #попробуем добавить тренд и сезонность
    # применяем seasonal_decompose
    # эта функция разложит ряд на трендовую, сезонную и шумовую составляющие
    decomposition = seasonal_decompose(data['PO'], model='additive', extrapolate_trend = 1) 
    data['PO_trend'] = decomposition.trend
    data['PO_trend'][0] = data['PO_trend'][1] - data['PO_trend'][1:].diff().mean()
    data['PO_seasonal'] = decomposition.seasonal
    data_grouped = data.groupby('month').mean()
    data['PO_seasonal'][0] = data_grouped['PO_seasonal'].iloc[0]

    for i in range(period):
        data = data.append(pd.Series(), ignore_index = True)
    
    
  
    # так как мы не знаем некоторые параметры в прогнозируемом году, необходимо их достроить 
    for i in range(-1*period,0):
        data['month'].iloc[i] = data['month'].iloc[i-1]%12 + 1
        data['year'].iloc[i] = data['year'].iloc[i-12] + 1
        data['date'].iloc[i] = str(int(data['year'].iloc[i])) + '-' + str(int(data['month'].iloc[i])) + '-01'
        
        data['PO_trend'].iloc[i] = data['PO_trend'].iloc[i-1] + data['PO_trend'].iloc[1:-1*period].diff().mean()
        data['PO_seasonal'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['PO_seasonal'].iloc[0]        
        data['temp_C'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['temp_C'].iloc[0]
        data['cos_month'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['cos_month'].iloc[0]
        
        data['po_average2'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['po_average2'].iloc[0]
        data['po_average_exp2'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['po_average_exp2'].iloc[0]
 
    data['date'] = pd.to_datetime(data['date'], dayfirst = True)
    data.index = data['date']
   

    # добавим столбец с фактом прошлого года
    data['fact_last_year'] = data['PO'].shift(freq=pd.DateOffset(years=1))
    # заполним пропуски первого года средним по месяцам
    for i in range(12):
        data['fact_last_year'][i] = data_grouped['PO'].iloc[i]
    if period > 12:
        for i in range(-1*period+12,0):
            data['fact_last_year'].iloc[i] = data_grouped[data_grouped.index == data['month'].iloc[i]]['PO'].iloc[0]
  
    
    # добавяем количество дней
    data['days'] = data['date'].apply(lambda x: pd.Period(str(x)).days_in_month)
    # добавляем количество рабочих дней и выходных
    data['work_days'] = data['date'].apply(count_work_days)
    data['week_ends'] = data['days'] - data['work_days']
    # добавим температуу в кельвинах
    data['temp_K'] = data['temp_C'] + 273
    data['prognos'] = data['PO_trend'] + data['PO_seasonal']
    
    
    return data
    
    
def count_work_days(date):
  """в данную функцию будем передавать дату, затем будем получать из нее
  календарь месяца в виде списка списков, где каждый список - это неделя с номерами дней в течении недели.
  На первой и последней неделе есть нули, если месяц начинается не с понедельника и закачивается е воскресеньем
  
  требует импорта модуля calendar
  """
  calendar_of_month = calendar.monthcalendar(date.year, date.month)
  work_days = 0
  for week in calendar_of_month:
    for work_day in week[:-2]:
      if work_day != 0:
         work_days += 1
  return work_days

In [4]:
# прогнозируемый период - кол-во месяцев
period = 12

path = r"C:\Гарант\прогнозирование\Население\test_folder\\"

features = [
            'cos_month',
            'days',
            'week_ends',
              ]
num_features = [
            'temp_C',
            'fact_last_year',
            'po_average2',
            'po_average_exp2', 
            'PO_trend',
            'PO_seasonal',
            'prognos'
               ]

cat_features = ['month',
                #'year'
               ]
scaler = StandardScaler()



# заберем все фалы из папок test-folder
files_list = []
for root, dirs, files in os.walk(path):  
    for filename in files:
        files_list.append(filename)

# загоним названия файлов экселя в список
files_excel = []
for file in files_list:
  if 'xlsx' in file and '~' not in file and 'prognos' not in file:
    files_excel.append(file)

# выберем модель
model = RandomForestRegressor(n_estimators = 250,
                                  min_samples_split = 5,
                                  min_samples_leaf = 1,
                                  max_features = 'sqrt',
                                  max_depth = 100,
                                  bootstrap = True)




predict = pd.DataFrame()
for file in files_excel:
    df = pd.read_excel(path + file)
    # создаем признаки, включая прогнозный год
    df = create_pred_features(df, period)
#    df = df[:-12]
    
    # нормализуем признаки
    df_for_pred = df.copy(deep = True)
    df_for_pred[num_features] = scaler.fit_transform(df_for_pred[num_features])
    df_for_pred = df_for_pred[num_features]
    df_for_pred[features] = df[features]
        
    # get dummies
    for category in cat_features:
        one_hot = pd.get_dummies(df[category])
        df_for_pred = df_for_pred.join(one_hot)


    X_train = df_for_pred[:-1*period]
    y_train = df['PO'][:-1*period]
    X_pred = df_for_pred[-1*period:]

    
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_pred)
    
    name = file.split('.')[0]
    predict[name+'_ПО_прогноз'] = y_pred
    predict.index = X_pred.index

#сохраним полученные данные в файл
predict.to_excel(excel_writer = path+'prognos_ml_next_year.xlsx')
